In [1]:
import requests
from bs4 import BeautifulSoup as bs

import pandas as pd

### Add headers (ethical obligation)

In [2]:
myUa = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0"
myInf = "VaishaliPandiyen PandiyenV@cardiff.ac.uk scraping for uni coursework"
myHeaders = {"User-Agent": f'{myUa}; {myInf}'}
myHeaders

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0; VaishaliPandiyen PandiyenV@cardiff.ac.uk scraping for uni coursework'}

### Run this to update:

In [3]:
aqw_sites = "https://airquality.gov.wales/air-pollution/monitoring-site-summary"
aqw_req_1 = requests.get(aqw_sites, myHeaders)

if aqw_req_1.status_code == 200:
    aqw_soup_1 = bs(aqw_req_1.content)

In [4]:
print("Now scraping:", aqw_soup_1.find('h1', class_ ='page-header')
      .text, "from", aqw_sites)


Now scraping: Monitoring site summary
 from https://airquality.gov.wales/air-pollution/monitoring-site-summary


In [5]:
site_links = aqw_soup_1.find('table', 
                class_='responsive-enabled table table-hover table-striped')
site_links = site_links.find('tbody').find_all('tr')

aqw_places = []
aqw_place_urls = []

for s in site_links:
    place = s.find('td').find('a').text
    aqw_places.append(place)
    
    url_bit = s.find('td').find('a').attrs['href']
    l = f'https://airquality.gov.wales{url_bit}'
    aqw_place_urls.append(l)

    
print(f"Air Quality Wales has air quality monitors in {len(aqw_places)} places")


Air Quality Wales has air quality monitors in 38 places


In [6]:
import datetime

In [76]:
pm25 = 0
no_pm25 = 0
place_list = []
source_list = []
pm25_list = []
pm25_places = []

lats = []
lons = []
site_type = []
comments = []
pm25_start = []

In [90]:
for l in aqw_place_urls:
    req = requests.get(l, myHeaders)
    soup = bs(req.content)
    
    all_els=''
    els = soup.find('div', class_='tab-content air-view-pollutant-content').find('table').find('tbody').find_all('tr')
    for el in els:
        e = el.find('td').text
        all_els+=f'{e}, '
        # if e not in el_list:
        #    el_list.append(e)
        #    el_dict[e]=[]     
        
    name = soup.find('section', class_='block block-waq-site-latest-data block-site-latest-data-block clearfix').find('div', class_='field field--name-body field--type-text-with-summary field--label-hidden field--item').find('h2').text
    place_list.append(name)
    
    source_list.append("AQW")
    
    if 'PM2.5' in all_els:
        pm25 += 1
        pm25_list.append('Yes')
        pm25_places.append(name)
    else: 
        no_pm25 += 1
        pm25_list.append('No')
    
    reqN = requests.get(f'{l}#siteinformation', myHeaders)
    soupN = bs(reqN.content)


In [77]:
for l in aqw_place_urls:
    data = soupN.find('div', id="siteinformation").find_all('table')[0].find_all('tr')
    site = data[1].find('td').text
    site_type.append(site)
    coord = data[3].find('td').text.split(',')
    lat = coord[0]
    lats.append(lat)
    lon = coord[1]
    lons.append(lon)
    comment = data[6].find('td').text
    comments.append(comment)
    
#     els = soupN.find('div', id="siteinformation").findAll('table')[1].find('tbody').findAll('tr')
#     for el in els:
#         if el.findAll('td')[0].text == "Particulates < 2.5um (hourly measured)":
#             pm25_start.append(el.findAll('td')[1].text)   

#     print(soupN.find_all('table')[3].find('tbody').findAll('tr')[0].findAll('td')[0])
#     print(data)

In [93]:
clean_place_list = [s.strip() for s in place_list]
clean_pm25_places = [s.strip() for s in pm25_places]

version = f'This AQW data is from {datetime.datetime.now()} {datetime.timezone.utc}'
print(version)
print(f'{pm25} places monitor PM2.5 and {no_pm25} do not')
# print(clean_pm25_places)

print(len(lats), len(lons), len(site_type), len(comments), len(pm25_start))

This AQW data is from 2023-05-11 11:59:04.414172 UTC
17 places monitor PM2.5 and 21 do not
38 38 38 38 38


In [85]:
pm25_start = ['NA', 'NA', 'NA', '04/05/2012', 'NA', 'NA', '20/10/2020', '12/08/2008', 'NA', '09/02/2010', 'NA', 'NA', 'NA', 'NA', 'NA', '05/06/2018', 'NA', '12/12/2008', '20/05/2022', '26/05/2022', '23/04/2008', '07/03/2014', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '15/01/2016', '20/12/2018', '20/09/2006', 'NA', 'NA', '01/01/2005', '01/10/2018', '22/07/2020', '29/11/2021']


In [99]:
auth = ['-', '-', '-', 'Local authority', '-', '-', 'Local authority', 'DEFRA', '-', 'DEFRA', '-', '-', '-', '-', '-', 'DEFRA', '-', 'DEFRA', 'Unknown', 'Unknown', 'DEFRA', 'Local authority', '-', '-', '-', '-', '-', '-', '-', 'Local authority', 'Local authority', 'DEFRA', '-', '-', 'Local authority', 'DEFRA', 'Local authority', 'Unknown']


In [100]:
d = {'Place': clean_place_list, 'Source': source_list, 'PM 2.5': pm25_list, 'Site type': site_type, 'Latitude': lats, 'Longitude': lons, 'Started recording PM 2.5 hourly': pm25_start, 'Monitoring Authority': auth, 'Comments': comments}
df = pd.DataFrame(columns=['Place', 'Source', 'PM 2.5', 'Site type', 'Latitude', 'Longitude', 'Started recording PM 2.5 hourly', 'Monitoring Authority', 'Comments'], data=d)
df


,Place,Source,PM 2.5,Site type,Latitude,Longitude,Started recording PM 2.5 hourly,Monitoring Authority,Comments
0,Aston Hill,AQW,No,Urban background,52.943644,-3.053116,NA,-,
1,Bridgend Park Street,AQW,No,Urban background,52.943644,-3.053116,NA,-,
2,Caerphilly Blackwood High Street,AQW,No,Urban background,52.943644,-3.053116,NA,-,
3,Caerphilly Fochriw,AQW,Yes,Urban background,52.943644,-3.053116,04/05/2012,Local authority,
4,Caerphilly Islwyn Road Wattsville,AQW,No,Urban background,52.943644,-3.053116,NA,-,
5,Caerphilly White Street,AQW,No,Urban background,52.943644,-3.053116,NA,-,
6,Cardiff Castle Street,AQW,Yes,Urban background,52.943644,-3.053116,20/10/2020,Local authority,
7,Cardiff Centre,AQW,Yes,Urban background,52.943644,-3.053116,12/08/2008,DEFRA,
8,Cardiff Newport Road,AQW,No,Urban background,52.943644,-3.053116,NA,-,
9,Chepstow A48,AQW,Yes,Urban background,52.943644,-3.053116,09/02/2010,DEFRA,


### Selenium (if needed)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Chrome()

lats = []
lons = []
site_type = []
comments = []

for l in aqw_place_urls:
    driver.get(l)
    # find and click the element that triggers the click event
    wait = WebDriverWait(driver, 10)
    link_text = "Site Information"
    try:
        link = wait.until(EC.element_to_be_clickable((By.XPATH, f"//a[contains(text(), '{link_text}')]")))
        link.click()
    except Exception as e:
        print(f"Error clicking link: {e}")
    site_info_div_id = "siteinformation"
    try:
        site_info_div = wait.until(EC.visibility_of_element_located((By.ID, site_info_div_id)))
#         print(f"Site information div contents: {site_info_div.text}")
    except Exception as e:
        print(f"Error finding site information div: {e}")
    # scrape the data from the dynamic content
    data = site_info_div.find_elements(By.TAG_NAME,'table')
    data2 = data[0].find_element(By.TAG_NAME,'tbody').find_elements(By.TAG_NAME,'tr')
    site = data2[3].find_element(By.TAG_NAME,'td').text
    site_type.append(site)
    coord_txt = data2[3].find_element(By.TAG_NAME,'td').text
    coord = coord_txt.split(',')
#     print(coord)
    lat = coord[0]
    lats.append(lat)
    lon = coord[1]
    lons.append(lon)
    comment = data2[6].find_element(By.TAG_NAME,'td').text
    comments.append(comment)
    # # close the browser
    driver.quit()
    
print(len(lats), len(lons), len(site_type), len(comments))
            

In [ ]:
'''
If we want data on all the monitored polutants: (WIP)

for l in aqw_place_urls: 
        for el in els:
            e = el.find('td').text
        for n in el_dict:
            el_dict[e].append("Yes")
        else:
            el_dict[e].append("No")
            
d = {'Place': clean_place_list, 'Source': source_list}

for e in el_list:
    d[e] = "dummy"
    el_subLists.append([])
    
df = pd.DataFrame(columns=['Place', 'Source', *el_list], data=d)
'''

In [ ]:
df2= df[df['Site type'] != "Urban Industrial"]
df3 = df2[df2['PM 2.5'] == "Yes"]

In [101]:
df.to_csv("wales_aqm.csv")

'https://airquality.gov.wales/air-pollution/site/AH'

['29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021',
 '29/11/2021']